# Can sentiment be predicted for Congolese French Facebook data?

Loading needed python packages

In [1]:
import csv
import pandas as pd 
import numpy as np  
import re  
import nltk  
from sklearn.datasets import load_files  
nltk.download('stopwords')  
import pickle  
from nltk.corpus import stopwords
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, numpy, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.preprocessing import text

import gensim 
from gensim.models import Word2Vec

import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /home/sije/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


Importing the data, ony message which have more than 3 words are selected.

In [2]:
df = pd.read_csv("/home/sije/Thesis/DRC_topics.csv", delimiter = ';')

df = df[df['event.message'].notnull()]
def get_length(message):
    """
    """
    message_size = len(message)
    return int(message_size)

convert_string = {'anger': 4, 'disgust': 1, 'fear': 7, 'happiness': 6,'neutral': 3,'sadness': 5,'surprise': 2}

def get_integer_emotion(emotion):
    """
    """
    new_integer = convert_string[emotion] 
    return int(new_integer)

df['size_message'] = df['event.message'].apply(get_length)
df['emotion_integer'] = df['event.emotion'].apply(get_integer_emotion)
df = df[df['size_message'] > 3]

/home/sije/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


This stemming function is applies a French steming on the data. Also unwanted data is removed

In [3]:
documents = []

from nltk.stem.snowball import FrenchStemmer

stemmer = FrenchStemmer()
def get_stemming(sen):  
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(sen))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.stem(word) for word in document]
    document = ' '.join(document)

    return(document)

In [4]:
df['message_stemmer'] = df['event.message'].apply(get_stemming)

The total number for each clas are printed. The zero class is almost three times bigger than the positive and negative class. 

In [5]:
print(df['event.polarity'].value_counts())

 0    209513
 1     73513
-1     60788
Name: event.polarity, dtype: int64


In [6]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df['message_stemmer'], df['event.polarity'])

In [7]:
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

3 data transformations are created word level tf-idf, ngram level tf-idf and characters level tf-idf

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['message_stemmer'])
xtrain_tfidf =  tfidf_vect.transform(X_train.apply(lambda x: np.str_(x)))
xvalid_tfidf =  tfidf_vect.transform(X_test.apply(lambda x: np.str_(x)))

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['message_stemmer'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(X_train.apply(lambda x: np.str_(x)))
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(X_test.apply(lambda x: np.str_(x)))

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['message_stemmer'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_train.apply(lambda x: np.str_(x)))
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_test.apply(lambda x: np.str_(x)))

Data is vecotrized, this the data without any extra datatransformation steps

In [9]:
count_vect = CountVectorizer()
count_vect.fit(df['message_stemmer'])
xtrain_count =  count_vect.transform(X_train.apply(lambda x: np.str_(x)))
xvalid_count =  count_vect.transform(X_test.apply(lambda x: np.str_(x)))

In [10]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    print(confusion_matrix(y_test,predictions))  
    print(classification_report(y_test,predictions))  
    print(accuracy_score(y_test,predictions))

In [32]:
# Naive Bayes on Count Vectors
print("NB, Count Vectors: ")
train_model(naive_bayes.MultinomialNB(), xtrain_count, y_train, xvalid_count)


# Naive Bayes on Word Level TF IDF Vectors
print("NB, WordLevel TF-IDF: ")
train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, y_train, xvalid_tfidf)

# Naive Bayes on Ngram Level TF IDF Vectors
print("NB, N-Gram Vectors: ")
train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, y_train, xvalid_tfidf_ngram)


# Naive Bayes on Character Level TF IDF Vectors
print("NB, CharLevel Vectors: ")
train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, y_train, xvalid_tfidf_ngram_chars)

NB, Count Vectors: 
[[ 3854  9239  2095]
 [ 4477 41017  7024]
 [ 1889  9799  6560]]
              precision    recall  f1-score   support

           0       0.38      0.25      0.30     15188
           1       0.68      0.78      0.73     52518
           2       0.42      0.36      0.39     18248

   micro avg       0.60      0.60      0.60     85954
   macro avg       0.49      0.46      0.47     85954
weighted avg       0.57      0.60      0.58     85954

0.5983549340344836
NB, WordLevel TF-IDF: 
[[ 1217 13404   567]
 [  459 50802  1257]
 [  345 15306  2597]]
              precision    recall  f1-score   support

           0       0.60      0.08      0.14     15188
           1       0.64      0.97      0.77     52518
           2       0.59      0.14      0.23     18248

   micro avg       0.64      0.64      0.64     85954
   macro avg       0.61      0.40      0.38     85954
weighted avg       0.62      0.64      0.54     85954

0.6354096377131955
NB, N-Gram Vectors: 
[[ 2401 

In [17]:
# Linear Classifier on Count Vectors
print("LR, Count Vectors: ")
train_model(linear_model.LogisticRegression(), xtrain_count, y_train, xvalid_count)


# Linear Classifier on Word Level TF IDF Vectors
print("LR, WordLevel TF-IDF: ")
train_model(linear_model.LogisticRegression(), xtrain_tfidf, y_train, xvalid_tfidf)


# Linear Classifier on Ngram Level TF IDF Vectors
print("LR, N-Gram Vectors: ")
train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, y_train, xvalid_tfidf_ngram)


# Linear Classifier on Character Level TF IDF Vectors
print("LR, CharLevel Vectors: ")
train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, y_train, xvalid_tfidf_ngram_chars)

LR, Count Vectors: 


/home/sije/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/sije/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/sije/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[[ 4915  8914  1181]
 [ 2495 46491  3578]
 [ 1534 11732  5114]]
              precision    recall  f1-score   support

           0       0.55      0.33      0.41     15010
           1       0.69      0.88      0.78     52564
           2       0.52      0.28      0.36     18380

   micro avg       0.66      0.66      0.66     85954
   macro avg       0.59      0.50      0.52     85954
weighted avg       0.63      0.66      0.62     85954

0.6575610210112386
LR, WordLevel TF-IDF: 
[[ 5082  8432  1496]
 [ 2015 47904  2645]
 [ 1308 11219  5853]]
              precision    recall  f1-score   support

           0       0.60      0.34      0.43     15010
           1       0.71      0.91      0.80     52564
           2       0.59      0.32      0.41     18380

   micro avg       0.68      0.68      0.68     85954
   macro avg       0.63      0.52      0.55     85954
weighted avg       0.66      0.68      0.65     85954

0.6845405682109035
LR, N-Gram Vectors: 
[[ 3174 10648  1188]
 [ 1420

In [ ]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, y_train, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: " + str(accuracy))

/home/sije/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [11]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, y_train, xvalid_count)
print("RF, Count Vectors: " + str(accuracy))

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, y_train, xvalid_tfidf)
print("RF, WordLevel TF-IDF: " + str(accuracy))

/home/sije/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[[ 7024  6357  1738]
 [ 4292 45367  2727]
 [ 2729 10048  5672]]
              precision    recall  f1-score   support

           0       0.50      0.46      0.48     15119
           1       0.73      0.87      0.79     52386
           2       0.56      0.31      0.40     18449

   micro avg       0.68      0.68      0.68     85954
   macro avg       0.60      0.55      0.56     85954
weighted avg       0.66      0.68      0.65     85954

0.6755124834213649
RF, Count Vectors: None


/home/sije/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[[ 7192  6269  1658]
 [ 2737 47254  2395]
 [ 2331 10191  5927]]
              precision    recall  f1-score   support

           0       0.59      0.48      0.53     15119
           1       0.74      0.90      0.81     52386
           2       0.59      0.32      0.42     18449

   micro avg       0.70      0.70      0.70     85954
   macro avg       0.64      0.57      0.59     85954
weighted avg       0.68      0.70      0.68     85954

0.702387323452079
RF, WordLevel TF-IDF: None


The Random Forest using TF-IDF results in an accuracy 0.702. Therefore it is the best scoring model. There is a difference between macro and micro average of 11 procent. This could be optimized by using a smaller set of the neutral class. 